In [114]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import numpy as np
from sklearn.metrics import classification_report


In [115]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")


In [116]:
model_name = "bert-base-uncased" 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

 62%|██████▏   | 1433/2295 [02:02<01:01, 13.94it/s]Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/saahil/Desktop/Coding_Projects/LLMS/classifier-based ft -llm glue/env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [117]:
for param in model.bert.parameters():
    param.requires_grad = False  # Freeze all layers

for param in model.bert.encoder.layer[-3:].parameters():
    param.requires_grad = True   # Unfreeze the last two layers

In [118]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [119]:
def preprocess_function(examples):
    # Tokenize the sentence pairs for BERT
    return tokenizer(
        examples['sentence1'], 
        examples['sentence2'], 
        truncation=True, 
        padding="max_length", 
        max_length=128
    )

In [121]:
dataset = load_dataset("glue", "mrpc")

In [122]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [123]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16=False,  # Disable mixed precision
    use_mps_device=True  # Use MPS if available
)

/Users/saahil/Desktop/Coding_Projects/LLMS/classifier-based ft -llm glue/env/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/saahil/Desktop/Coding_Projects/LLMS/classifier-based ft -llm glue/env/lib/python3.9/site-packages/transformers/training_args.py:2179: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


In [1]:
from transformers import EvalPrediction
def compute_metrics(p: EvalPrediction):
    predictions = torch.argmax(torch.tensor(p.predictions), dim=-1)
    labels = torch.tensor(p.label_ids)
    accuracy = (predictions == labels).float().mean().item()
    return {"accuracy": accuracy}

/Users/saahil/Desktop/Coding_Projects/LLMS/classifier-based ft -llm glue/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/saahil/Desktop/Coding_Projects/LLMS/classifier-based ft -llm glue/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [125]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [126]:
trainer.train()


  0%|          | 12/22950 [00:00<28:09, 13.57it/s]

{'loss': 0.7697, 'grad_norm': 9.011730194091797, 'learning_rate': 9.995642701525055e-05, 'epoch': 0.02}


  0%|          | 22/22950 [00:01<27:10, 14.06it/s]

{'loss': 0.6097, 'grad_norm': 2.7702386379241943, 'learning_rate': 9.991285403050109e-05, 'epoch': 0.04}


  0%|          | 32/22950 [00:02<27:13, 14.03it/s]

...

                                                     
100%|██████████| 22950/22950 [38:28<00:00, 10.47it/s]

{'eval_loss': 1.348001480102539, 'eval_accuracy': 0.8529411554336548, 'eval_runtime': 2.7068, 'eval_samples_per_second': 150.73, 'eval_steps_per_second': 18.841, 'epoch': 50.0}


100%|██████████| 22950/22950 [38:30<00:00,  9.93it/s]

{'train_runtime': 2310.0483, 'train_samples_per_second': 79.392, 'train_steps_per_second': 9.935, 'train_loss': 0.05217436972030999, 'epoch': 50.0}


TrainOutput(global_step=22950, training_loss=0.05217436972030999, metrics={'train_runtime': 2310.0483, 'train_samples_per_second': 79.392, 'train_steps_per_second': 9.935, 'total_flos': 1.2063641888256e+16, 'train_loss': 0.05217436972030999, 'epoch': 50.0})

In [127]:
results = trainer.evaluate()
print("Evaluation Results:", results)

100%|██████████| 51/51 [00:02<00:00, 24.44it/s]

Evaluation Results: {'eval_loss': 0.3803345859050751, 'eval_accuracy': 0.8602941036224365, 'eval_runtime': 2.2411, 'eval_samples_per_second': 182.057, 'eval_steps_per_second': 22.757, 'epoch': 50.0}


In [128]:
model.save_pretrained("./glue_mrpc_bert_model")
tokenizer.save_pretrained("./glue_mrpc_bert_tokenizer")

('./glue_mrpc_bert_tokenizer/tokenizer_config.json',
 './glue_mrpc_bert_tokenizer/special_tokens_map.json',
 './glue_mrpc_bert_tokenizer/vocab.txt',
 './glue_mrpc_bert_tokenizer/added_tokens.json',
 './glue_mrpc_bert_tokenizer/tokenizer.json')

In [129]:
predictions = trainer.predict(tokenized_datasets["validation"])

100%|██████████| 51/51 [00:02<00:00, 24.41it/s]


In [130]:
predictions = trainer.predict(tokenized_datasets["validation"])

100%|██████████| 51/51 [00:02<00:00, 24.35it/s]


In [131]:
import numpy as np
from sklearn.metrics import classification_report

labels = np.array(tokenized_datasets["validation"]["label"])
logits = predictions.predictions
y_pred = np.argmax(logits, axis=-1)

In [132]:
# Classification Report
class_report = classification_report(labels, y_pred)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.71      0.76       129
           1       0.88      0.93      0.90       279

    accuracy                           0.86       408
   macro avg       0.85      0.82      0.83       408
weighted avg       0.86      0.86      0.86       408



In [133]:
# Load the fine-tuned model and tokenizer
model_name = "./glue_mrpc_bert_model"  # Path to the saved fine-tuned model
tokenizer_name = "./glue_mrpc_bert_tokenizer"  # Path to the saved tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [134]:
sentence1 = "Ram is very happy"
sentence2 = "He is sad"


In [135]:
inputs = tokenizer(sentence1, sentence2, return_tensors="pt", padding="max_length", max_length=128, truncation=True)


In [136]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}


In [137]:
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)



In [138]:
# Print the prediction
predicted_class = predictions.item()
class_names = ["Not Equivalent", "Equivalent"]
print(f"Predicted class: {predicted_class} ({class_names[predicted_class]})")

Predicted class: 0 (Not Equivalent)
